### Import Libraries

In [1]:
import pandas as pd

### Read data for model

In [4]:
train_df = pd.read_csv("CBIS-DDSM_Clean_Data/train_descriptions.csv")
test_df = pd.read_csv("CBIS-DDSM_Clean_Data/test_descriptions.csv")

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2864 entries, 0 to 2863
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   breast_density    2864 non-null   int64 
 1   breast_side       2864 non-null   object
 2   image_view        2864 non-null   object
 3   abnormality_type  2864 non-null   object
 4   pathology         2864 non-null   object
 5   series_uid        2864 non-null   object
 6   images_new_paths  2864 non-null   object
 7   image_type        2864 non-null   object
dtypes: int64(1), object(7)
memory usage: 179.1+ KB


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2864 entries, 0 to 2863
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   breast_density    2864 non-null   int64 
 1   breast_side       2864 non-null   object
 2   image_view        2864 non-null   object
 3   abnormality_type  2864 non-null   object
 4   pathology         2864 non-null   object
 5   series_uid        2864 non-null   object
 6   images_new_paths  2864 non-null   object
 7   image_type        2864 non-null   object
dtypes: int64(1), object(7)
memory usage: 179.1+ KB


In [10]:
train_df.iloc[0]

breast_density                                                      3
breast_side                                                      LEFT
image_view                                                         CC
abnormality_type                                                 mass
pathology                                                   MALIGNANT
series_uid          1.3.6.1.4.1.9590.100.1.2.342386194811267636608...
images_new_paths    1.3.6.1.4.1.9590.100.1.2.342386194811267636608...
image_type                                      full mammogram images
Name: 0, dtype: object